In [1]:
import pandas as pd

df = pd.read_csv('data/name_gender_filtered.csv')
unique_chars = set()

for name in df['Name']:
    unique_chars.update(name)
unique_chars = sorted(list(unique_chars))
unique_chars = ''.join(unique_chars)
print(unique_chars, len(unique_chars))

abcdefghijklmnopqrstuvwxyz 26


In [2]:
import torch

n_letters = len(unique_chars)
def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters)
    for char_idx, char in enumerate(name):
        letter_idx = unique_chars.find(char)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor

gen2num = {'F':0, 'M':1}
num2gen = {0:'F', 1:'M'}

print(nameToTensor('leo'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])


In [3]:
# tensor check

shuffled_df = df.sample(frac=1).reset_index(drop=True)

for index, row in shuffled_df.iterrows():
    input_tensor = nameToTensor(row['Name'])
    target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)
    
    print(row['Name'])
    print(input_tensor)
    print(row['Gender'], gen2num[row['Gender']])
    print(target_tensor)
    break

celene
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
F 0
tensor([0])


In [4]:
from myRNN import *

n_hidden = 32
rnn_model = MyRNN(n_letters, n_hidden, 2)

In [6]:
import torch
from torch.optim import Adam, SGD

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_model.parameters(), lr=0.0001)

rnn_model.train()
for epoch_idx in range(100):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)

    total_loss = 0.
    correct_predictions = 0
    total_predictions = 0

    for index, row in shuffled_df.iterrows():
        input_tensor = nameToTensor(row['Name'])
        target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)

        hidden = rnn_model.get_hidden()

        rnn_model.zero_grad()

        for char_idx in range(input_tensor.size()[0]):
            char_tensor = input_tensor[char_idx]
            output, hidden = rnn_model(char_tensor[None,:], hidden)

        loss = loss_fn(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted_index = torch.argmax(output, 1)
        correct_predictions += (predicted_index == target_tensor).sum().item()
        total_predictions += 1

    average_loss = total_loss / total_predictions
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Epoch: {epoch_idx}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch: 0, Loss: 0.3729, Accuracy: 83.62%
Epoch: 1, Loss: 0.3707, Accuracy: 83.76%
Epoch: 2, Loss: 0.3689, Accuracy: 83.88%
Epoch: 3, Loss: 0.3682, Accuracy: 84.03%
Epoch: 4, Loss: 0.3677, Accuracy: 83.90%
Epoch: 5, Loss: 0.3672, Accuracy: 83.93%
Epoch: 6, Loss: 0.3660, Accuracy: 84.14%
Epoch: 7, Loss: 0.3655, Accuracy: 84.14%
Epoch: 8, Loss: 0.3649, Accuracy: 84.14%
Epoch: 9, Loss: 0.3649, Accuracy: 84.20%
Epoch: 10, Loss: 0.3638, Accuracy: 84.29%
Epoch: 11, Loss: 0.3628, Accuracy: 84.28%
Epoch: 12, Loss: 0.3627, Accuracy: 84.23%
Epoch: 13, Loss: 0.3612, Accuracy: 84.34%
Epoch: 14, Loss: 0.3603, Accuracy: 84.50%
Epoch: 15, Loss: 0.3587, Accuracy: 84.40%
Epoch: 16, Loss: 0.3580, Accuracy: 84.55%
Epoch: 17, Loss: 0.3563, Accuracy: 84.77%
Epoch: 18, Loss: 0.3543, Accuracy: 84.70%
Epoch: 19, Loss: 0.3533, Accuracy: 84.70%
Epoch: 20, Loss: 0.3513, Accuracy: 85.14%
Epoch: 21, Loss: 0.3503, Accuracy: 84.91%
Epoch: 22, Loss: 0.3478, Accuracy: 85.12%
Epoch: 23, Loss: 0.3461, Accuracy: 85.00%
Ep

In [ ]:
test_name = 'elsa'
test_name_tensor = nameToTensor(test_name)

rnn_model.eval()
hidden = rnn_model.get_hidden()
for char_idx in range(len(test_name)):
    char_tensor = test_name_tensor[char_idx]
    output, hidden = rnn_model(char_tensor[None,:],hidden)
    
predicted_index = torch.argmax(output, 1).item()
print(num2gen[predicted_index])